In [1]:
import time
import psutil
import os

# Definimos una función que nos permita medir el uso de recursos (tiempo de ejecución, uso de CPU y memoria empleada) del código antes de comenzar
def comienzo_medicion():
    proceso = psutil.Process(os.getpid())
    estado_inicial = {
        'cpu_inicio' : proceso.cpu_percent(interval=None),
        'memoria_inicio' : proceso.memory_info().rss, 
        'tiempo_inicio' : time.time()
    }
    return estado_inicial

# Definimos una función que nos permita medir el uso de recursos (tiempo de ejecución, uso de CPU y memoria empleada) del código después de haber ejecutado el código
def final_medicion(estado_inicial):
    proceso = psutil.Process(os.getpid())
    tiempo_fin = time.time()
    fin_cpu = proceso.cpu_percent(interval=None)
    fin_memoria = proceso.memory_info().rss

    tiempo_ejecucion = tiempo_fin - estado_inicial['tiempo_inicio']
    cpu_usada = fin_cpu - estado_inicial['cpu_inicio']
    memoria_usada = (fin_memoria - estado_inicial['memoria_inicio'])

    print(f"El tiempo de ejecución de todo el código es de {tiempo_ejecucion:.2f} segundos.")
    print(f"La CPU empleada todo el código es de {cpu_usada:.2f} %.")
    print(f"La memoria utilizada en todo el código es de {memoria_usada:.2f} MB.")

estado_inicial = comienzo_medicion() # Empezamos con la medición de los recursos

In [2]:
import os 
import re

# Establecemos el directorio de trabajo
os.chdir("/home/mpnavarrete")

# Definimos el directorio en el que se ubican las secuencias de estudio
dir_sequences = "/home/mpnavarrete/sequences"

# Obtenemos la lista de archivos .fastq en el directorio
ficheros_fastq = [f for f in os.listdir(dir_sequences) if f.endswith('.fastq')]

# Definimos la secuencia de inserción (IS6110) y su longitud
is6110_inicio = "tgaaccgccccggcatgtccggagactccagtt"
longitud_target_inicio = len(is6110_inicio)

# Convertimos la secuencia de inserción a mayúsculas
is6110_inicio = is6110_inicio.upper()
#print(is6110_inicio)

In [3]:
from Bio import SeqIO
from Bio.SeqUtils import nt_search

# Definimos una función para buscar el patrón inicio en cada read
def buscar_patron_inicio(fichero):
    ruta_completa = os.path.join(dir_sequences, fichero) # Obtenemos la ruta completa del archivo de estudio
    registros = SeqIO.parse(ruta_completa, "fastq") # Vamos guardando los reads de cada archivo FASTQ
    resultados = {} # Almacenamos en un diccionario los resultados generados

    # Bucle para trabajar con cada read
    for idx, record in enumerate(registros, 1):
        fq_text = str(record.seq) # Convertimos cada read en un "string" de texto
        buscar_patron_inicio = nt_search(fq_text, is6110_inicio) # Buscamos la IS6110 dentro de cada read
        detectar_reads_inicio = len(buscar_patron_inicio) != 0 # Comprobamos que IS6110 se ha encontrado

        # Condición de si se encuentra IS6110
        if detectar_reads_inicio:
            fastq_detectadas_inicio = fq_text # Guardamos la secuencia en la que se ha encontrado
            posicion_secuencia_inicio_en_fastq = buscar_patron_inicio # Guardamos la posición en la que se ha encontrado
            
            # Generamos un nuevo componente en el diccionario con la información anterior
            resultados[idx] = {
                'fastq_detectadas_inicio': fastq_detectadas_inicio,
                'posicion_secuencia_inicio_en_fastq': posicion_secuencia_inicio_en_fastq
            }

    return resultados

# Procesamos cada archivo en ficheros_fastq
lista_data_frames = {} # Generamos un nuevo diccionario para poder almacenar los resultados obtenidos

# Recorremos todos los FASTQ que tenemos
for fichero in ficheros_fastq:
    nombre_archivo = os.path.basename(fichero) # Obtenemos el nombre del archivo FASTQ
    resultados = buscar_patron_inicio(fichero) # Con la función, buscamos la IS6110
    lista_data_frames[nombre_archivo] = resultados # Guardamos los resultados utilizando el nombre del FASTQ

# Ahora lista_data_frames contiene los resultados para cada archivo .fastq, con los nombres de archivo sin la ruta completa


In [4]:
# Comprobamos que la cantidad de archivos FASTQ encontrados es la misma del estudio
print(f"Archivos FASTQ encontrados: {len(ficheros_fastq)}")
print(ficheros_fastq)

Archivos FASTQ encontrados: 12
['20ID00942_S12_R1_001.fastq', '20ID00940_S10_R2_001.fastq', '20ID00940_S10_R1_001.fastq', '20ID00937_S7_R2_001.fastq', '20ID00942_S12_R2_001.fastq', '20ID00941_S11_R2_001.fastq', '20ID00938_S8_R1_001.fastq', '20ID00941_S11_R1_001.fastq', '20ID00937_S7_R1_001.fastq', '20ID00939_S9_R2_001.fastq', '20ID00939_S9_R1_001.fastq', '20ID00938_S8_R2_001.fastq']


In [5]:
import pickle

# Guardamos lista_data_frames en un archivo
with open("lista_data_frames.pkl", "wb") as f:
    pickle.dump(lista_data_frames, f)

In [6]:
# Cargamos list_data_frames desde el archivo creado
with open('lista_data_frames.pkl', 'rb') as f:
    lista_data_frames = pickle.load(f)

# Volvemos a definir la longitud de la secuencia IS6110
longitud_target_inicio = len("TGAACCGCCCCGGCATGTCCGGAGACTCCAGTT")

# Creamos otro diccionario para almacenar resultados
lista_resultados = {}

# Procesamos caa elemento de la lista
for key, df in lista_data_frames.items():
    pre_target_inicio = [] # Esta lista guardará la secuencia desde el inicio hasta el final del IS6110
    pre_6_nucleotidos = [] # Esta lista guardará los 6 nucleótidos previos al comienzo de la IS6110
    post_target_inicio = [] # Esta lista guardará la parte de la secuencia posterior a la IS6110

    for indice_fastq, values in df.items():
        secuencia = values['fastq_detectadas_inicio']
        is6110_inicio = "TGAACCGCCCCGGCATGTCCGGAGACTCCAGTT"
        indicador_comienzo_is = secuencia.find(is6110_inicio) # Nos devuelve la posición en el que IS6110 empieza dentro del read. En caso de que no esté, devuelve -1
        longitud_secuencia = len(secuencia)

        if indicador_comienzo_is != -1: # Cuando IS6110 sí está (es diferente de -1)
            pre_target_inicio.append(str(secuencia[:indicador_comienzo_is + longitud_target_inicio])) # Obtenemos la secuencia desde el principio hasta el final de la IS

            # Extraemos los 6 nucleótidos justo antes de la IS
            if indicador_comienzo_is >= 6:
                pre_6_nucleotidos.append(str(secuencia[indicador_comienzo_is - 6:indicador_comienzo_is]))
            else:
                pre_6_nucleotidos.append('NA') # Si no hay 6 nucleótidos previos al comienzo de la IS6110

            # Extraemos la parte después de la IS6110
            if indicador_comienzo_is + longitud_target_inicio <= longitud_secuencia:
                post_target_inicio.append(str(secuencia[indicador_comienzo_is + longitud_target_inicio:longitud_secuencia]))
            else:
                post_target_inicio.append('NA') # Si no queda secuencia restante después de haber sacado la parte previa a la IS6110 + la parte de la IS6110

    # Generamos un nuevo componente en el diccionario con la información anterior
    lista_resultados[key] = {
        'pre_target_inicio': pre_target_inicio,
        'pre_6_nucleotidos': pre_6_nucleotidos,
        'post_target_inicio': post_target_inicio
    }

# Guardamos lista_resultados en un archivo
with open('lista_resultados.pkl', 'wb') as f:
    pickle.dump(lista_resultados, f)

In [7]:
# Nos aseguramos de que lista_resultados contiene todos los resultados
print(f"Archivos en lista_resultados: {len(lista_resultados)}")
print(lista_resultados.keys())

Archivos en lista_resultados: 12
dict_keys(['20ID00942_S12_R1_001.fastq', '20ID00940_S10_R2_001.fastq', '20ID00940_S10_R1_001.fastq', '20ID00937_S7_R2_001.fastq', '20ID00942_S12_R2_001.fastq', '20ID00941_S11_R2_001.fastq', '20ID00938_S8_R1_001.fastq', '20ID00941_S11_R1_001.fastq', '20ID00937_S7_R1_001.fastq', '20ID00939_S9_R2_001.fastq', '20ID00939_S9_R1_001.fastq', '20ID00938_S8_R2_001.fastq'])


In [8]:
#print(type(lista_resultados))

In [9]:
import pandas as pd

# Contamos cuántas veces aparece cada cadena de 6 nucleótidos previa a la IS6110 en nuestros reads
lista_pre_6_nucleotidos = {}

for key, value in lista_resultados.items():
    # Verificamos si hay resultados en cada archivo
    if 'pre_6_nucleotidos' in value and len(value['pre_6_nucleotidos']) > 0: # Miramos si cada valor contiene datos sobre los 6 nucleótidos previos a IS6110
        data_frame_6_nucleotidos = pd.DataFrame(value['pre_6_nucleotidos'], columns=['fragmento']) # Creamos un DataFrame con la lista de los fragmentos de 6 nucleótidos
        sorted_table = data_frame_6_nucleotidos['fragmento'].value_counts().reset_index().rename(columns={'index': 'fragmento', 'fragmento': 'count'}) # Contamos el número de veces que aparece cada fragmento
        lista_pre_6_nucleotidos[key] = sorted_table
    else:
        print(f"Advertencia: No se encontraron nucleótidos previos a la IS en {key}")

# Verificamos el resultado final
print(f"Total de archivos procesados: {len(lista_pre_6_nucleotidos)}")
lista_pre_6_nucleotidos

Total de archivos procesados: 12


{'20ID00942_S12_R1_001.fastq':      count  count
 0   TCTGAT      9
 1   AGGAGG      8
 2       NA      8
 3   TCGGAG      7
 4   CCATCA      6
 5   AGTGTC      6
 6   CGCCTA      5
 7   CCAACG      5
 8   CCTGGA      5
 9   TTTGCG      4
 10  TCACAA      4
 11  CGGAAA      2
 12  CTAACG      1
 13  TCAGAC      1
 14  GGGGCC      1,
 '20ID00940_S10_R2_001.fastq':      count  count
 0   GGGGCC     11
 1   CCAACG      8
 2   CGCCTA      5
 3       NA      5
 4   TCAGAC      5
 5   TTTGCG      5
 6   AGGAGG      4
 7   TCGGAG      4
 8   TCTGAT      4
 9   CCATCA      3
 10  TCACAA      3
 11  AGTGTC      1
 12  CGGAAA      1
 13  TCATCA      1,
 '20ID00940_S10_R1_001.fastq':      count  count
 0   CCATCA     11
 1   TTTGCG     10
 2   CGGAAA      8
 3       NA      8
 4   TCACAA      6
 5   AGGAGG      6
 6   TCTGAT      5
 7   CCAACG      5
 8   TCAGAC      5
 9   AGTGTC      4
 10  CGCCTA      4
 11  GGGGCC      4
 12  TCGGAG      3,
 '20ID00937_S7_R2_001.fastq':      count  count
 0  

In [10]:
# Ordenamos la parte que hay previa a IS6110 hasta el final de la IS según su longitud
lista_pre_target_inicio = {} # Diccionario para almacenar los DataFrames ordenados y sin duplicados

for key, value in lista_resultados.items():
    if 'pre_target_inicio' in value:
        data_frame_pre_target = pd.DataFrame(value['pre_target_inicio'], columns=['secuencia_pre_target']) # Creamos un DataFrame a partir de 'pre_target_inicio'
        
        data_frame_pre_target = data_frame_pre_target.drop_duplicates(subset=['secuencia_pre_target']) # Eliminamos las filas duplicadas basándonos en la columna 'secuencia_pre_target'
        
        data_frame_pre_target['longitud_cadena'] = data_frame_pre_target['secuencia_pre_target'].str.len() # Calculamos la longitud de cada cadena
        
        data_frame_pre_target = data_frame_pre_target.sort_values(by='longitud_cadena', ascending=False) # Ordenamos el DataFrame por la longitud de las cadenas en orden descendente
        
        data_frame_pre_target = data_frame_pre_target.drop(columns=['longitud_cadena']) # Eliminamos la columna de longitud 
        
        lista_pre_target_inicio[key] = data_frame_pre_target # Guardamos el DataFrame ordenado en el diccionario
    else:
        print(f"Advertencia: No se encontraron secuencias en pre_target_inicio para {key}")

print(f"Total de archivos ordenados: {len(lista_pre_target_inicio)}")
lista_pre_target_inicio

Total de archivos ordenados: 12


{'20ID00942_S12_R1_001.fastq':                                  secuencia_pre_target
 52  GCCCCGGCACGTCCATTCGAGTATCCCGGTCGCCGTGACATTGACC...
 20  GCCAAAGCGGGCCGCCAGGTGTTCAAGGACTATGTCGCGTTCCCGG...
 51  GCTTTAGGCCCAGCTGGAGCCGACGGCGCTGTCGGTTTGTGCCATG...
 47  AGGTGAGAAGGACCGGCCGGCGTGACTTGATAGGAGCGTGGCTTTC...
 42  ACCTAGGCGGCGGCAATATCGGCAGCTTCAACCTGGGCAGCGGTAA...
 ..                                                ...
 33              ATCATGAACCGCCCCGGCATGTCCGGAGACTCCAGTT
 27              GGAGTGAACCGCCCCGGCATGTCCGGAGACTCCAGTT
 23              GAAATGAACCGCCCCGGCATGTCCGGAGACTCCAGTT
 14              AGACTGAACCGCCCCGGCATGTCCGGAGACTCCAGTT
 41               GAGTGAACCGCCCCGGCATGTCCGGAGACTCCAGTT
 
 [67 rows x 1 columns],
 '20ID00940_S10_R2_001.fastq':                                  secuencia_pre_target
 36  GGCCCCGGCACGTCCATTCGAGTATCCCGGTCGCCGTGACATTGAC...
 6   ACGAATACGACATGACCACAGACTCCGGCACACCCCTCAGCGAATT...
 59  GAATACGACATGACCACAGACTCCGGCACACCCCTCAGCGAATTCT...
 54  ATCGCCGGTGACACTGGCACGCCAGGGTG

In [11]:
# Ordenamos la parte que hay posterior a IS6110 según su longitud
lista_post_target = {} # Diccionario para almacenar los DataFrames ordenados y sin duplicados de post_target

for key, value in lista_resultados.items():
    data_frame_pre_target = pd.DataFrame(value['pre_target_inicio'], columns=['secuencia_pre_target']) # Creamos un DataFrame a partir de 'pre_target_inicio'
    data_frame_post_target = pd.DataFrame(value['post_target_inicio'], columns=['secuencia_post_target']) # Creamos un DataFrame a partir de 'post_target_inicio'
    
    data_frame_post_target = data_frame_post_target.drop_duplicates(subset=['secuencia_post_target']).reset_index(drop=True) # Eliminamos las filas duplicadas en función de 'secuencia_post_target'
    
    # Nos aseguramos de que data_frame_pre_target y data_frame_post_target tengan la misma longitud, en caso contrario el DataFrame más largo se recortará
    if len(data_frame_pre_target) != len(data_frame_post_target):
        min_length = min(len(data_frame_pre_target), len(data_frame_post_target))
        data_frame_pre_target = data_frame_pre_target.iloc[:min_length].reset_index(drop=True)
        data_frame_post_target = data_frame_post_target.iloc[:min_length].reset_index(drop=True)
    
    longitud_cadena = data_frame_pre_target['secuencia_pre_target'].str.len() # Calculamos la longitud de cada cadena en 'secuencia_pre_target'
    
    data_frame_post_target = data_frame_post_target.iloc[longitud_cadena.argsort()[::-1]].reset_index(drop=True) # Ordenamos data_frame_post_target según la longitud de la cadena en orden descendente

    lista_post_target[key] = data_frame_post_target # Guardamos el DataFrame ordenado en el diccionario

lista_post_target

{'20ID00942_S12_R1_001.fastq':                                 secuencia_post_target
 0                                                CTTG
 1   CTGGGGAGGGATGGGGTCATGTCAGGTGGTTCATCGAGGAGGTACC...
 2                                        CTTGGAAAGGAT
 3                                       CTTGGAAAGGATG
 4                                  CTTGGAAAGGATGGGGTG
 5   CTTGGAAAGGATGGGGTCATGTCAGGTGGTTCATCGAGGAGGTACC...
 6                                 CTTGGAAAGGATGGGGTCA
 7                               CTTGGAAAGGATGGGGTCATG
 8                     CTTGGAAAGGATGGGGTCATGTCAGGTGGTT
 9                  CTTGGAAAGGATGGGGTCATGTCAGGTGGTTCAT
 10                                   CTTGGAAAGGATGGGG
 11           CTTGGAAAGGATGGGGTCATGTCAGGTGGTTCATCGAGGA
 12       CTTGGAAAGGATGGGGTCATGTCAGGTGGTTCATCGAGGAGGTA
 13  CTTGGAAAGGATGGGGTCATGTCAGGTGGTTCATCGAGGAGGTACC...
 14                                 CTTGGAAAGGATGGGGTC
 15                                  CTTGGAAAGGATGGGGT
 16                                

In [12]:
# Creación de archivos Excel
# Creamos un diccionario para almacenar todos los DataFrames
data_frames = {
    "pre_6_nucleotidos": lista_pre_6_nucleotidos,
    "pre_target_inicio": lista_pre_target_inicio,
    "post_target": lista_post_target
}

# Definimos una función para guardar DataFrames en un archivo Excel
def guardar_dataframes_en_excel(nombre_archivo, data):
    with pd.ExcelWriter(nombre_archivo, engine='openpyxl') as writer:

        for documento, df in data.items():
            sheet_name = documento # Creamos un nombre para cada hoja
            df.to_excel(writer, sheet_name=sheet_name, index=False) # Escribimos el DataFrame en una hoja
    print(f"Archivo {nombre_archivo} creado exitosamente.")

# Guardamos cada conjunto de DataFrames en tres archivos Excel por separado
guardar_dataframes_en_excel('resultados_pre_6_nucleotidos.xlsx', lista_pre_6_nucleotidos)
guardar_dataframes_en_excel('resultados_pre_target_inicio.xlsx', lista_pre_target_inicio)
guardar_dataframes_en_excel('resultados_post_target.xlsx', lista_post_target)

Archivo resultados_pre_6_nucleotidos.xlsx creado exitosamente.
Archivo resultados_pre_target_inicio.xlsx creado exitosamente.
Archivo resultados_post_target.xlsx creado exitosamente.


In [13]:
# Parte del código realmente no esencial, puramente informativa
lista_resultados_en_data_frame = {} # Creamos un nuevo DataFrame a partir de los resultados para juntar la información

for key, value in lista_resultados.items():
    df_pre = pd.DataFrame(value['pre_target_inicio'], columns=['Pre'])
    df_seis = pd.DataFrame(value['pre_6_nucleotidos'], columns=['Seis.Nucleotidos'])
    df_post = pd.DataFrame(value['post_target_inicio'], columns=['Post'])
    
    length_pre = df_pre['Pre'].str.len()
    
    df_resultado_muestra = pd.DataFrame({
        'Pre': df_pre['Pre'],
        'Length': length_pre,
        'Seis.Nucleotidos': df_seis['Seis.Nucleotidos'],
        'Post': df_post['Post']
    })
    
    df_resultado_muestra = df_resultado_muestra.sort_values(by='Length', ascending=False)
    lista_resultados_en_data_frame[key] = df_resultado_muestra

lista_resultados_en_data_frame

{'20ID00942_S12_R1_001.fastq':                                                   Pre  Length  \
 52  GCCCCGGCACGTCCATTCGAGTATCCCGGTCGCCGTGACATTGACC...     148   
 20  GCCAAAGCGGGCCGCCAGGTGTTCAAGGACTATGTCGCGTTCCCGG...     145   
 61  AGGTGAGAAGGACCGGCCGGCGTGACTTGATAGGAGCGTGGCTTTC...     140   
 51  GCTTTAGGCCCAGCTGGAGCCGACGGCGCTGTCGGTTTGTGCCATG...     140   
 47  AGGTGAGAAGGACCGGCCGGCGTGACTTGATAGGAGCGTGGCTTTC...     140   
 ..                                                ...     ...   
 14              AGACTGAACCGCCCCGGCATGTCCGGAGACTCCAGTT      37   
 33              ATCATGAACCGCCCCGGCATGTCCGGAGACTCCAGTT      37   
 43              CCTATGAACCGCCCCGGCATGTCCGGAGACTCCAGTT      37   
 27              GGAGTGAACCGCCCCGGCATGTCCGGAGACTCCAGTT      37   
 41               GAGTGAACCGCCCCGGCATGTCCGGAGACTCCAGTT      36   
 
    Seis.Nucleotidos                                               Post  
 52           CGCCTA                                                  A  
 20           CCAACG        

In [14]:
import os

# Definimos una función para escribir un archivo FASTA ya que es el formato necesario para el BLASTn
def escribir_fasta(secuencias, nombre_archivo):
    with open(nombre_archivo, 'w') as f:
        for i, secuencia in enumerate(secuencias, 1):
            f.write(f">Secuencia_{i}\n")
            f.write(f"{secuencia}\n")

# Directorio de salida para los archivos FASTA
output_dir = "/home/mpnavarrete/fasta_para_blastn"
# Creamos el directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iteramos sobre los datos en lista_pre_target_inicio y creamos los archivos FASTA a partir de esos datos
for key, df in lista_pre_target_inicio.items():
    secuencias_pre = df['secuencia_pre_target'].dropna().tolist()
    nombre_archivo_pre = os.path.join(output_dir, f"{key}_pre_target_inicio.fasta")
    escribir_fasta(secuencias_pre, nombre_archivo_pre)

In [15]:
import os
import subprocess
import pandas as pd
from Bio import SeqIO

# Ruta al genoma de referencia de Mycobacterium tuberculosis H37Rv
genome_fasta = "/home/mpnavarrete/GCA_000195955.2_ASM19595v2_genomic.fna"
genome_gff = "/home/mpnavarrete/GCF_000195955.2_ASM19595v2_genomic.gff"

# Creamos la base de datos BLAST si no existe
blast_db = "/home/mpnavarrete/mtb_h37rv_db"
if not os.path.exists(blast_db + ".nhr"):
    subprocess.run(["makeblastdb", "-in", genome_fasta, "-dbtype", "nucl", "-out", blast_db])

# Directorio de salida para los resultados de BLAST
blast_output_dir = "/home/mpnavarrete/resultados_blastn_forward"
# Creamos el directorio de salida si no existe
if not os.path.exists(blast_output_dir):
    os.makedirs(blast_output_dir)

# Directorio para los resultados anotados en formato .txt
annotated_output_dir = "/home/mpnavarrete/blastn_forward_anotados"
# Creamos el directorio de salida si no existe
if not os.path.exists(annotated_output_dir):
    os.makedirs(annotated_output_dir)

# Nombres de las columnas para los resultados del BLASTn
blast_columns = [
    "Query", "Subject ID", "% Identity", "Alignment Length", "Mismatches",
    "Gap Opens", "Q. Start", "Q. End", "S. Start", "S. End", "E-value", "Bit Score"
]

# Definimos una función para leer el archivo GFF y almacenar la información relevante
def parse_gff(gff_file):
    gff_data = {}
    current_gene = None

    with open(gff_file, 'r') as gff:
        for line in gff:
            if line.startswith('#'):
                continue
            parts = line.strip().split('\t')
            if len(parts) != 9:
                continue
            seqname, source, feature, start, end, score, strand, frame, attributes = parts
            attr_dict = {kv_pair.split('=')[0]: kv_pair.split('=')[1] for kv_pair in attributes.split(';')}

            # Si estamos ante un gen, guardamos la información que nos interesa en función del ID del gen
            if feature == "gene":
                gene_id = attr_dict.get('ID', 'unknown')
                gff_data[gene_id] = {
                    'seqname': seqname,
                    'start': int(start),
                    'end': int(end),
                    'strand': strand,
                    'cds': []
                }
                current_gene = gene_id # Se establece el gen actual para añadirle CDS (región de codificación)

            # Si estamos ante un caso de una región codificante, se agrega también sus datos correspondientes
            elif feature == "CDS" and current_gene:
                product = attr_dict.get('product', 'No description')
                gff_data[current_gene]['cds'].append({
                    'seqname': seqname,
                    'start': int(start),
                    'end': int(end),
                    'strand': strand,
                    'description': product
                })

    return gff_data

gff_data = parse_gff(genome_gff) # Leemos y almacenamos la información del archivo GFF

# Definimos una función para obtener las secuencias del archivo FASTA
def get_sequences_from_fasta(fasta_file):
    sequences = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences[record.id] = str(record.seq)
    return sequences

# Ejecutaremos BLASTn para cada archivo FASTA y anotaremos sus resultados
for key in lista_pre_target_inicio.keys():
    fasta_file = os.path.join(output_dir, f"{key}_pre_target_inicio.fasta") # Definimos la ruta en la que están guardados los archivos 
    blast_output_file = os.path.join(blast_output_dir, f"{key}_blastn_results.txt") # Definimos la ruta en la que se guardan los resultados de BLAST
    annotated_output_file = os.path.join(annotated_output_dir, f"{key}_blastn_results_annotated.txt") # Definimos la ruta en la que se guardan los resultados de BLAST anotados
    
    sequences = get_sequences_from_fasta(fasta_file) # Obtenemos las secuencias de los archivos FASTA

    blast_df = pd.read_csv(blast_output_file, sep='\t', header=None, names=blast_columns) # Leemos el archivo de resultados de BLASTn

    blast_df['Query'] = blast_df['Query'].apply(lambda x: sequences.get(x, 'Unknown')) # Reemplazamos "Query ID" con la secuencia correspondiente para que sea más informativo para la persona que lo vaya a mirar

    # Definimos una función para buscar la anotación basada en la posición en el GFF
    def find_annotation(row, gff_data):
        for gene_id, gene_info in gff_data.items():
            if (gene_info['seqname'] == row['Subject ID'] and
                gene_info['start'] <= row['S. Start'] <= gene_info['end']):
                for cds in gene_info['cds']:
                    if cds['start'] <= row['S. Start'] <= cds['end']:
                        return gene_id, cds['description']
        return "No annotation", "No description"

    blast_df[['Gene', 'Description']] = blast_df.apply(lambda row: pd.Series(find_annotation(row, gff_data)), axis=1) # Añadimos dos nuevas columnas con la anotación y la descripción

    print(blast_df.head()) # Verificamos si las anotaciones y descripciones se están añadiendo correctamente

    blast_df.to_csv(annotated_output_file, sep='\t', index=False) # Guardamos los resultados anotados en un nuevo archivo .txt

print("Proceso de anotación y guardado en formato .txt completado.")


                                               Query  Subject ID  % Identity  \
0  GCCCCGGCACGTCCATTCGAGTATCCCGGTCGCCGTGACATTGACC...  AL123456.3       100.0   
1  GCCCCGGCACGTCCATTCGAGTATCCCGGTCGCCGTGACATTGACC...  AL123456.3       100.0   
2  GCCCCGGCACGTCCATTCGAGTATCCCGGTCGCCGTGACATTGACC...  AL123456.3       100.0   
3  GCCCCGGCACGTCCATTCGAGTATCCCGGTCGCCGTGACATTGACC...  AL123456.3       100.0   
4  GCCCCGGCACGTCCATTCGAGTATCCCGGTCGCCGTGACATTGACC...  AL123456.3       100.0   

   Alignment Length  Mismatches  Gap Opens  Q. Start  Q. End  S. Start  \
0               115           0          0         1     115   1895765   
1                34           0          0       115     148   2365413   
2                34           0          0       115     148   3551229   
3                33           0          0       116     148    889021   
4                33           0          0       116     148   1543306   

    S. End       E-value  Bit Score           Gene           Description  

In [16]:
import os
import pandas as pd

# Directorio para los resultados anotados en formato .txt
annotated_output_dir = "/home/mpnavarrete/blastn_forward_anotados"

txt_files = [f for f in os.listdir(annotated_output_dir) if f.endswith('_blastn_results_annotated.txt')] # Obtenemos la lista de los archivos .txt en el directorio anotado

# Vamos a convertir cada archivo .txt a un archivo .xlsx
for txt_file in txt_files:
    txt_file_path = os.path.join(annotated_output_dir, txt_file)
    excel_file_path = os.path.join(annotated_output_dir, txt_file.replace('.txt', '.xlsx'))
    
    df = pd.read_csv(txt_file_path, sep='\t') # Leemos el archivo .txt
    
    df.to_excel(excel_file_path, index=False) # Guardamos ese archivo .txt como archivo .xlsx
    
    print(f"Convertido {txt_file} a {excel_file_path}")

print("Conversión de archivos completada.")

Convertido 20ID00940_S10_R1_001.fastq_blastn_results_annotated.txt a /home/mpnavarrete/blastn_forward_anotados/20ID00940_S10_R1_001.fastq_blastn_results_annotated.xlsx
Convertido 20ID00942_S12_R2_001.fastq_blastn_results_annotated.txt a /home/mpnavarrete/blastn_forward_anotados/20ID00942_S12_R2_001.fastq_blastn_results_annotated.xlsx


Convertido 20ID00937_S7_R1_001.fastq_blastn_results_annotated.txt a /home/mpnavarrete/blastn_forward_anotados/20ID00937_S7_R1_001.fastq_blastn_results_annotated.xlsx
Convertido 20ID00941_S11_R2_001.fastq_blastn_results_annotated.txt a /home/mpnavarrete/blastn_forward_anotados/20ID00941_S11_R2_001.fastq_blastn_results_annotated.xlsx
Convertido 20ID00940_S10_R2_001.fastq_blastn_results_annotated.txt a /home/mpnavarrete/blastn_forward_anotados/20ID00940_S10_R2_001.fastq_blastn_results_annotated.xlsx
Convertido 20ID00938_S8_R2_001.fastq_blastn_results_annotated.txt a /home/mpnavarrete/blastn_forward_anotados/20ID00938_S8_R2_001.fastq_blastn_results_annotated.xlsx
Convertido 20ID00941_S11_R1_001.fastq_blastn_results_annotated.txt a /home/mpnavarrete/blastn_forward_anotados/20ID00941_S11_R1_001.fastq_blastn_results_annotated.xlsx
Convertido 20ID00939_S9_R1_001.fastq_blastn_results_annotated.txt a /home/mpnavarrete/blastn_forward_anotados/20ID00939_S9_R1_001.fastq_blastn_results_annotated.xls

In [17]:
import os
import pandas as pd

# Directorio para los resultados anotados en formato .xlsx
annotated_output_dir = "/home/mpnavarrete/blastn_forward_anotados"

# Directorio de salida para el archivo combinado
combined_output_dir = "/home/mpnavarrete"
# Creamos el directorio de salida si no existe
if not os.path.exists(combined_output_dir):
    os.makedirs(combined_output_dir)

xlsx_files = [f for f in os.listdir(annotated_output_dir) if f.endswith('.xlsx')] # Obtenemos la lista de archivos .xlsx en el directorio anotado

# Ruta del archivo combinado en el nuevo directorio
combined_excel_path = os.path.join(combined_output_dir, 'resultados_blastn_forward_anotados_combined.xlsx')

# Definimos una función para verificar si un archivo es un archivo .xlsx válido
def is_valid_excel_file(filepath):
    try:
        pd.read_excel(filepath, engine='openpyxl')
        return True
    except Exception as e:
        print(f"Error leyendo el archivo {filepath}: {e}")
        return False

# Creamos un archivo Excel con múltiples pestañas (combinacion de varios Excel)
with pd.ExcelWriter(combined_excel_path, engine='openpyxl') as writer:
    for xlsx_file in xlsx_files:
        xlsx_file_path = os.path.join(annotated_output_dir, xlsx_file)
        if is_valid_excel_file(xlsx_file_path):
            sheet_name = os.path.splitext(xlsx_file)[0][:31]  # Nombre de la pestaña truncado a 31 caracteres (de otra forma da error)
            df = pd.read_excel(xlsx_file_path, engine='openpyxl')
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"Añadido {xlsx_file} como pestaña {sheet_name}")
        else:
            print(f"El archivo {xlsx_file_path} no es un archivo .xlsx válido y será omitido.")

print("Combinación de archivos .xlsx completada.")

Añadido 20ID00941_S11_R2_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00941_S11_R2_001.fastq_blas
Añadido 20ID00937_S7_R1_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00937_S7_R1_001.fastq_blast
Añadido 20ID00937_S7_R2_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00937_S7_R2_001.fastq_blast
Añadido 20ID00940_S10_R2_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00940_S10_R2_001.fastq_blas
Añadido 20ID00942_S12_R1_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00942_S12_R1_001.fastq_blas
Añadido 20ID00939_S9_R2_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00939_S9_R2_001.fastq_blast
Añadido 20ID00942_S12_R2_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00942_S12_R2_001.fastq_blas
Añadido 20ID00938_S8_R2_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00938_S8_R2_001.fastq_blast
Añadido 20ID00941_S11_R1_001.fastq_blastn_results_annotated.xlsx como pestaña 20ID00941_S11_R1_001.fastq_blas
Añadido 20ID00

In [18]:
final_medicion(estado_inicial)

El tiempo de ejecución de todo el código es de 231.75 segundos.
La CPU empleada todo el código es de 0.00 %.
La memoria utilizada en todo el código es de 5039304704.00 MB.
